In [1]:
!pip install tqdm

In [2]:
import torch
import wandb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, ConcatDataset, Subset
from tqdm.notebook import tqdm


In [3]:
wandb.login(key='72a114321dd97dbf11db7b15eb05b2660c2faa94')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
torch.cuda.is_available()

True

In [5]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [6]:
def prepare_dataset(batch_size, transform):
    # Define the directory containing your dataset
    train_dir = '/kaggle/input/inaturalist-12k/inaturalist_12K/train'
    test_dir = '/kaggle/input/inaturalist-12k/inaturalist_12K/val'


    train_dataset = ImageFolder(train_dir, transform=transform)
    test_dataset = ImageFolder(test_dir, transform=transform)

    validation_ratio = 0.2
    class_labels = [label for _, label in train_dataset]

    class_id = defaultdict(list)
    for idx, label in enumerate(class_labels):
        class_id[label].append(idx)

    train_indices = []
    val_indices = []

    for y, ids in class_id.items():
        num_samples = len(ids)
        val_samples = int(validation_ratio * num_samples)
        np.random.shuffle(ids)  # Shuffle indices for random selection
        train_indices.extend(ids[val_samples:])
        val_indices.extend(ids[:val_samples])

    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
    
    val_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=val_sampler)
    
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
    
    return train_loader, val_loader, test_loader

In [7]:
def train_per_epoch(model, train_loader, loss_func, optimizer, epoch):
    model.train()  # Set the model to training mode
    
    train_loss = 0.0
    correct_ans = 0
    num_samples = 0
        
    for data in tqdm(train_loader):
        inputs, labels = data[0].to(device), data[1].to(device)
            
        optimizer.zero_grad()
            
        outputs = model(inputs)
        loss = loss_func(outputs, labels)
            
        loss.backward()
        optimizer.step()
            
        train_loss += loss.item() * inputs.size(0)
        z, predicted = torch.max(outputs, 1)
        correct_ans += (predicted == labels).sum().item()
        num_samples += labels.size(0)
            
        
    epoch_loss = train_loss / num_samples
    epoch_accuracy = correct_ans / num_samples
    print(f'Train Size {num_samples}')
    return epoch_loss, epoch_accuracy

In [8]:
def val_per_epoch(model, val_loader, loss_func, optimizer, epoch):
    model.eval()
    
    correct_ans = 0
    num_samples = 0
    val_loss = 0.0
    
    with torch.no_grad():
        for data in tqdm(val_loader):
            inputs, labels = data[0].to(device), data[1].to(device)

            outputs = model(inputs)
            loss = loss_func(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            z, predicted = torch.max(outputs, 1)
            correct_ans += (predicted == labels).sum().item()
            num_samples += labels.size(0)
            
    epoch_val_loss = val_loss / num_samples
    epoch_val_accuracy = correct_ans / num_samples
    print(f'Val Size {num_samples}')
    return epoch_val_loss, epoch_val_accuracy
        

In [9]:
# Function to train the model
def train_model(model, train_loader, val_loader, loss_func, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        train_loss, train_acc = train_per_epoch(model, train_loader, loss_func, optimizer, epoch)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}')
        val_loss, val_acc = val_per_epoch(model, val_loader, loss_func, optimizer, epoch)
        print(f'Epoch {epoch+1}/{num_epochs}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}')
        #wandb.log({'train_loss': train_loss, 'train_accuracy': train_acc, 'val_loss' : val_loss, 'val_accuracy' : val_acc, 'epochs' : epoch + 1})
        

In [10]:
# Function to train the model
def test_model(model, test_loader, loss_func, optimizer):
    model.eval()
    
    correct_ans = 0
    num_samples = 0
    val_loss = 0.0
    
    with torch.no_grad():
        for data in tqdm(val_loader):
            inputs, labels = data[0].to(device), data[1].to(device)

            outputs = model(inputs)
            loss = loss_func(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            z, predicted = torch.max(outputs, 1)
            correct_ans += (predicted == labels).sum().item()
            num_samples += labels.size(0)
            
    test_loss = val_loss / num_samples
    test_acc = correct_ans / num_samples
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}')

In [11]:
# Fine-tuning function
def fine_tune(model, strategy='feature_extraction'):
    if strategy == 'feature_extraction':
        # Freeze all layers except the final classification layer
        for name, param in model.named_parameters():
            if "fc" not in name:  # Skip parameters of the final fully connected layer
                param.requires_grad = False
    elif strategy == 'fine_tuning_all':
        # Unfreeze all layers and train the entire model
        for param in model.parameters():
            param.requires_grad = True
    elif strategy == 'layer_wise_fine_tuning':
        # Unfreeze and fine-tune only a subset of layers (e.g., only top layers)
        for i, param in enumerate(model.parameters()):
            if i < 6:  # Freeze the first 6 layers (adjust as needed)
                param.requires_grad = False
                
    return model
    

In [12]:
weights= models.ResNet101_Weights.DEFAULT
auto_transforms = weights.transforms()

#Prepare Data
train_loader, val_loader, test_loader = prepare_dataset(32, auto_transforms)

In [13]:
print(auto_transforms)

ImageClassification(
    crop_size=[224]
    resize_size=[232]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)


In [14]:
# Load pre-trained model
model = models.resnet101(weights = weights)

# Modify the final classification head
num_classes = 10  # Number of classes in iNaturalist dataset
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, num_classes)

# Define loss function and optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
model = fine_tune(model, strategy="feature_extraction")
model.to(device)

train_model(model, train_loader, val_loader, loss_func, optimizer, num_epochs=5)

Downloading: "https://download.pytorch.org/models/resnet101-cd907fc2.pth" to /root/.cache/torch/hub/checkpoints/resnet101-cd907fc2.pth
100%|██████████| 171M/171M [00:01<00:00, 158MB/s]


  0%|          | 0/250 [00:00<?, ?it/s]

Train Size 8000
Epoch 1/5, Loss: 1.1527, Accuracy: 0.6736


  0%|          | 0/63 [00:00<?, ?it/s]

Val Size 1999
Epoch 1/5, Val Loss: 0.8338, Val Accuracy: 0.7544


...

Val Size 1999
Epoch 4/5, Val Loss: 0.6257, Val Accuracy: 0.8074


  0%|          | 0/250 [00:00<?, ?it/s]

Train Size 8000
Epoch 5/5, Loss: 0.4964, Accuracy: 0.8462


  0%|          | 0/63 [00:00<?, ?it/s]

Val Size 1999
Epoch 5/5, Val Loss: 0.6462, Val Accuracy: 0.7989


In [15]:
test_model(model, test_loader,loss_func, optimizer)

  0%|          | 0/63 [00:00<?, ?it/s]

Test Loss: 0.6462, Test Accuracy: 0.7989
